### Analyzing the World Bank's Twitter Feed, Judy Yang, DAT10 Project
### Part 1. Collect Data

In [15]:
pwd

u'/Users/judyyang/Documents/GA_DS_course/Final_Project/notebooks'

In [16]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
from time import sleep

pd.options.display.max_columns = 50
pd.options.display.max_rows= 50
pd.options.display.width= 120

#### Authentication

In [17]:
consumer_key = "Mdl5ArrrGvj1HHHzalPDiQlfd" # Use your own key. To get a key https://apps.twitter.com/
consumer_secret = "77q1xZWFhLiutXlnNef3tRu1tfvU1Qa42X7HU3FBe4uKvdRw40"

auth = tweepy.OAuthHandler(consumer_key=consumer_key, consumer_secret=consumer_secret)

api = tweepy.API(auth)

#### Find WB related accounts on Twitter

In [18]:
#Q: How to retrieve wb related accounts automatically
#wb_accts = api.get_user(id="WBG_Poverty")
#data = pd.DataFrame(wb_accts)

In [19]:
#manual is OK
#Q: how to write multiple api requests without getting timed out

#open file with WB twiter handles

data = pd.read_csv('./data/wb_twitter_ids.csv', delim_whitespace=True, header=None, names=['id'])
wb_ids=data[3:38]
#wb_ids=data[:38]
#start with a shorter set of ids
#Q: How to select line by id value not by index
wb_ids.head()

,id
3,WorldBankVideos
4,WBPubs
5,WBG_Education
6,wb_research
7,WBG_Health


In [20]:
### Store Results in a Data Frame

def process_results(results):
    id_list = [tweet.id for tweet in results]
    data_set = pd.DataFrame(id_list, columns=["id"])

    # Processing Tweet Data

    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["favorite_count"] = [tweet.favorite_count for tweet in results]
    data_set["source"] = [tweet.source for tweet in results]

    # Processing User Data
    data_set["user_id"] = [tweet.author.id for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_name"] = [tweet.author.name for tweet in results]
    data_set["user_created_at"] = [tweet.author.created_at for tweet in results]
    data_set["user_description"] = [tweet.author.description for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_friends_count"] = [tweet.author.friends_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]

    return data_set

Ask at office hours:

https://github.com/tweepy/tweepy/issues/617

I agree with @scubjwu. To me it happened when the connection went off. I have "fixed" by catching ReadTimeoutError and ConnectionError within the Timeout except.
except (Timeout, ssl.SSLError, ReadTimeoutError, ConnectionError) as exc:

Note that ReadTimeoutError belongs to urllib3 packages
from requests.exceptions import Timeout, ConnectionError
from requests.packages.urllib3.exceptions import ReadTimeoutError

In [21]:
# In this example, the handler is time.sleep(15 * 60),
# but you can of course handle it in any way you want.

def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)

In [22]:
#create a dataframe of all known WB twitter related handles
#Q: how to avoid a read time out, where to put the sleep timer?

all_wb = pd.DataFrame()

for wid in wb_ids.id :
    results = []
    for tweet in limit_handled(tweepy.Cursor(api.user_timeline, id=wid).items()):
        results.append(tweet)
    #print tweet    
    print wid, len(results), "start sleeping"
    sleep(61) 
    print "done sleeping"
    #print results
    add=process_results(results)
    all_wb=all_wb.append(add, ignore_index=True)

WorldBankVideos 2724 start sleeping
done sleeping


TweepError: Twitter error response: status code = 429

In [ ]:
all_wb.user_screen_name.value_counts()

### Save the Data

In [ ]:
#Save results as a json file
#version WorldBank_request_01feb_2016a has WBG_Climate, WBG_Poverty, worldbankdata
#all_wb.to_json('./data/WorldBank_request_01feb_2016a')